In [1]:
import polars as pl

data = pl.read_parquet("data/DECENTRATHON_3.0.parquet")

In [3]:
data.null_count()

transaction_id,transaction_timestamp,card_id,expiry_date,issuer_bank_name,merchant_id,merchant_mcc,mcc_category,merchant_city,transaction_type,transaction_amount_kzt,original_amount,transaction_currency,acquirer_country_iso,pos_entry_mode,wallet_type,__index_level_0__
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,2470036,2470036,0,2469297,0,0,11531433,0,0,2653412,4673494,0
